## Export Wav2vec2 model to ONNX

In [1]:
import transformers
from transformers import AutoTokenizer, Wav2Vec2ForCTC
from torchaudio.models.wav2vec2.utils import import_huggingface_model

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
imported = import_huggingface_model(original) 

config.json: 100%|██████████| 1.84k/1.84k [00:00<00:00, 9.36MB/s]
/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:365: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
pytorch_model.bin: 100%|██████████| 1.26G/1.26G [01:51<00:00, 11.4MB/s]
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at airesearch/wav2vec2-large-xlsr-53-th were not u

In [3]:
imported.eval()

Wav2Vec2Model(
  (feature_extractor): FeatureExtractor(
    (conv_layers): ModuleList(
      (0): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
      )
      (1-4): 4 x ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (5-6): 2 x ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
      )
    )
  )
  (encoder): Encoder(
    (feature_projection): FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=1024, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (transformer): Transformer(
      (pos_conv_embed): ConvolutionalPositionalEmbedding(
        (

In [23]:
import torch.onnx
from pathlib import Path
input_size = 160000
AUDIO_MAXLEN = input_size
MODEL_OUTDIR = Path("/models/wav2vec2/1/")
MODEL_OUTDIR.mkdir(parents=True, exist_ok=True)

In [24]:
dummy_input = torch.randn(1, input_size, requires_grad=True)


In [25]:
torch.onnx.export(
        imported,         # model being run
         dummy_input,       # model input (or a tuple for multiple inputs)
         f"{MODEL_OUTDIR}/model.onnx",       # where to save the model
         export_params=True,  # store the trained parameter weights inside the model file
         opset_version=14,    # the ONNX version to export the model to
         do_constant_folding=True,  # whether to execute constant folding for optimization
         input_names = ['input'],   # the model's input names
         output_names = ['output'], # the model's output names
         dynamic_axes={
            'input' : {
                0: 'batch_size',
                1: 'input_sequence'
                },    
            'output' : {
                0: 'batch_size',
                1: 'output_sequence'
                }
            
            })

## Inference test

In [8]:
!wget https://www.dropbox.com/s/9kpeh8eodshcqhj/common_voice_th_23646850.wav?dl=1
!mv common_voice_th_23646850.wav?dl=1 ${ROOT}/sound.wav
!wget https://huggingface.co/airesearch/wav2vec2-large-xlsr-53-th/raw/main/vocab.json 
!mv vocab.json ${ROOT}/vocab.json


--2024-01-23 15:44:41--  https://www.dropbox.com/s/9kpeh8eodshcqhj/common_voice_th_23646850.wav?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.84.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9kpeh8eodshcqhj/common_voice_th_23646850.wav [following]
--2024-01-23 15:44:45--  https://www.dropbox.com/s/dl/9kpeh8eodshcqhj/common_voice_th_23646850.wav
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc90027e405434cb4b55460bc627.dl.dropboxusercontent.com/cd/0/get/CL5n94jmUsw5bzMB2rmocpC-yk5jqRElBegT7gg_BLU_wQbVYwaeQ0TBXlfCHbV3SyQ9x7ZFOtBD165dDUNo165BrtJx3N19pJi9a2aeuT_TCt1LpBwyrnidXjpiFGbyMQwoGNgW6PuA8v8cAKv0m5O2/file?dl=1# [following]
--2024-01-23 15:44:46--  https://uc90027e405434cb4b55460bc627.dl.dropboxusercontent.com/cd/0/get/CL5n94jmUsw5bzMB2rmocpC-yk5jqRElBegT7gg_BLU_wQbVY

In [9]:
with open(f"/workspace/vocab.json","r",encoding="utf-8-sig") as f:
  d = eval(f.read())

In [10]:
print(d)

{'ฑ': 0, 'ๅ': 1, 'ก': 2, 'ง': 3, 'ฒ': 4, 'ะ': 5, '๊': 6, '้': 7, 'ฌ': 8, 'ซ': 9, 'ด': 10, 'ฯ': 11, 'ใ': 12, 'ึ': 13, 'ญ': 14, '่': 15, 'า': 16, 'ฤ': 17, '๋': 18, 'อ': 19, 'ฬ': 20, 'ท': 21, 'โ': 22, 'ภ': 23, 'ย': 24, '็': 25, 'ล': 26, 'ุ': 27, 'เ': 28, 'ฮ': 29, 'ฝ': 30, 'ป': 31, 'ี': 32, 'บ': 33, 'ฐ': 34, 'ต': 35, 'ถ': 36, 'ศ': 37, 'ฟ': 38, 'ณ': 39, 'ห': 40, 'ร': 41, 'พ': 43, 'ฆ': 44, 'ั': 45, 'ค': 46, 'ว': 47, 'ฏ': 48, 'จ': 49, 'แ': 50, 'ม': 51, 'ฎ': 52, 'ฉ': 53, '์': 54, 'ษ': 55, 'ำ': 56, 'ผ': 57, 'ข': 58, 'ไ': 59, 'ู': 60, 'ื': 61, 'น': 62, 'ช': 63, 'ิ': 64, 'ธ': 65, 'ฃ': 66, 'ส': 67, '|': 42, '[UNK]': 68, '[PAD]': 69}


## Inference

In [11]:
import onnx
import onnxruntime

In [13]:
import numpy as np
import soundfile as sf
from scipy.io import wavfile
import scipy.signal as sps
import os
from pythainlp.util import normalize

In [15]:
input_size = 100000
new_rate = 16000
AUDIO_MAXLEN = input_size
ort_session = onnxruntime.InferenceSession(f'{MODEL_OUTDIR}/model.onnx') # load onnx model


In [16]:
res = dict((v,k) for k,v in d.items())
res[69]="[PAD]"
res[68]="[UNK]"

In [17]:
def _normalize(x): #
  """You must call this before padding.
  Code from https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/processor.py#L101
  Fork TF to numpy
  """
  # -> (1, seqlen)
  mean = np.mean(x, axis=-1, keepdims=True)
  var = np.var(x, axis=-1, keepdims=True)
  return np.squeeze((x - mean) / np.sqrt(var + 1e-5))

In [18]:
def remove_adjacent(item): # code from https://stackoverflow.com/a/3460423
  nums = list(item)
  a = nums[:1]
  for item in nums[1:]:
    if item != a[-1]:
      a.append(item)
  return ''.join(a)

In [19]:
def asr(path):
    """
    Code from https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/notebooks/wav2vec2_onnx.ipynb
    Fork TF to numpy
    """
    sampling_rate, data = wavfile.read(path)
    samples = round(len(data) * float(new_rate) / sampling_rate)
    new_data = sps.resample(data, samples)
    speech = np.array(new_data, dtype=np.float32)
    speech = _normalize(speech)[None]
    padding = np.zeros((speech.shape[0], AUDIO_MAXLEN - speech.shape[1]))
    speech = np.concatenate([speech, padding], axis=-1).astype(np.float32)
    ort_inputs = {"input": speech}
    ort_outs = ort_session.run(None, ort_inputs)
    prediction = np.argmax(ort_outs, axis=-1)
    # Text post processing
    _t1 = ''.join([res[i] for i in list(prediction[0][0])])
    return normalize(''.join([remove_adjacent(j) for j in _t1.split("[PAD]")]))

In [20]:
FILENAME = "/workspace/sound.wav"



In [21]:
asr(FILENAME)

/tmp/ipykernel_3164/2770446858.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, data = wavfile.read(path)


'บริษัท|เรา|จะ|ต้อง|ปรับตัว|เพื่อ|ใช้งาน|เทคโนโลยี|เหล่านี้'